In [ ]:
from sense_hat import SenseHat
import time, os
import numpy as np

LABEL = "move_circle"
SAMPLES = 50
FREQ_HZ = 50
DELAY = 1.0 / FREQ_HZ

save_dir = f"./motion_data/{LABEL}"
os.makedirs(save_dir, exist_ok=True)

sense = SenseHat()

while True:
    input("Press Enter to record 1 second...")
    data = []
    for _ in range(SAMPLES):
        acc = sense.get_accelerometer_raw()
        gyro = sense.get_gyroscope_raw()
        data.append([
            acc['x'], acc['y'], acc['z'],
            gyro['x'], gyro['y'], gyro['z']
        ])
        time.sleep(DELAY)
    filename = f"{LABEL}_{int(time.time())}.npy"
    np.save(os.path.join(save_dir, filename), np.array(data))
    print(f"Saved {filename}")


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(300,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=1,
    epochs=15
)


In [ ]:
interpreter = tf.lite.Interpreter(model_path="motion_model.tflite") interpreter.allocate_tensors() input_details = interpreter.get_input_details() output_details = interpreter.get_output_details()
samples = [read_imu_sample() for _ in range(50)] input_data = np.expand_dims(np.array(samples).flatten().astype(np.
float32), axis=0)
interpreter.set_tensor(input_details[0][’index’], input_data) interpreter.invoke() output = interpreter.get_tensor(output_details[0][’index’])[0]
